<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/wisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install moviepy pyannote-audio

In [2]:
import numpy as np
from pyannote.audio import Pipeline, Audio
from pyannote.core import Segment
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

In [3]:
!pip install moviepy

In [4]:
from moviepy.editor import VideoFileClip

In [5]:
video_path = "./9d4fc932d442ef2ffb17d0a8086390e6_1725887318814_0.mp4"
audio_path = '.'.join(video_path.split('.')[:-1]) + '.wav'
video = VideoFileClip(video_path)
video.audio.write_audiofile(audio_path, ffmpeg_params=["-ac", "1"])  #モノラルで出力

MoviePy - Writing audio in ./9d4fc932d442ef2ffb17d0a8086390e6_1725887318814_0.wav


MoviePy - Done.


In [6]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-m9nem_1_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-m9nem_1_
  Resolved https://github.com/openai/whisper.git to commit 279133e3107392276dc509148da1f41bfb532c7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
import whisper
import json, re

In [8]:
model = whisper.load_model("small")

In [18]:
result = model.transcribe(audio_path, verbose=True, fp16=False, language="ja")

[00:00.000 --> 00:10.000] どうもこんにちは
[00:10.000 --> 00:12.000] ありがとうございます
[00:12.000 --> 00:19.000] ごめん 記者の時間いただきましてどうもありがとうございます
[00:20.000 --> 00:22.000] おはよう
[00:24.000 --> 00:29.000] では早速質問をさせていただきたいと思います
[00:29.000 --> 00:31.000] ちょっとだけお願いします
[00:31.000 --> 00:49.000] では早速なんですけれども
[00:49.000 --> 00:55.000] まずフリーをどうでしたでしょうか
[00:56.000 --> 01:02.000] 今ちょっと回線がフリーズして聞こえなかったんですけど
[01:02.000 --> 01:07.000] フリーを導入されたのはいつ頃でしたでしょうか
[01:07.000 --> 01:19.000] 今から6年前で2018年の4月ですね
[01:19.000 --> 01:23.000] すごい そんな前からだったんですね
[01:23.000 --> 01:25.000] はい
[01:25.000 --> 01:31.000] その当時他の例えばクラウド会計のものとかって
[01:31.000 --> 01:33.000] 検討されましたでしょうか
[01:33.000 --> 01:41.000] マネーフォアードとフリーとどっちにしようかなという話はちょっと
[01:41.000 --> 01:44.000] 私の中だけでありましたけども
[01:45.000 --> 01:53.000] フリーの方がどうも私はその会計に詳しい人間ではないので
[01:53.000 --> 02:03.000] 初心者向きだという話がちょっとしまったのレビューでありましたので
[02:03.000 --> 02:08.000] じゃあフリーの方いいかなと思ってフリーにしました
[02:09.000 --> 02:11.000] 少し出しました
[02:11.000 --> 02:18.000] 価格とかはその時って検討されましたか企画というか
[02:18.000 --> 0

In [10]:
segments = result['segments']
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか|でしょうか|ましたか|されましたか|いつ頃')
INTERVIEW = []
for seg in segments:
  count = len(prog.findall(seg['text']))
  if count > 0: print(seg['text'])
  _seg = {'start': seg['start'], 'end': seg['end'], 'text': seg['text'], 'count': count}
  INTERVIEW.append(_seg)

ちょっとだけお願いします
まずフリーをどうでしたでしょうか
フリーを導入されたのはいつ頃でしたでしょうか
検討されましたでしょうか
価格とかはその時って検討されましたか企画というか
ご自身でやっていらっしゃるビジネスにですかね
キャンセルした方がよろしいでしょうか
そうですか


In [11]:
segments = result['segments']
embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device='cpu')
embeddings = np.zeros(shape=(len(segments), 192))

duration = video.duration
embeddings = []


for i, segment in enumerate(segments):
    audio = Audio()
    start = segment["start"]
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(audio_path, clip)
    print(waveform[0])
    _enbed = embedding_model(waveform[None])
    #print(_enbed[0])
    embeddings.append(_enbed[0])

embeddings = np.nan_to_num(embeddings)


tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0009, -0.0007, -0.0003])
tensor([-0.0004, -0.0005, -0.0005,  ...,  0.0006,  0.0000, -0.0003])
tensor([ 3.3569e-04,  6.1035e-05, -2.4414e-04,  ...,  2.7161e-03,
         3.9673e-03,  5.0964e-03])
tensor([-0.0007, -0.0009, -0.0014,  ..., -0.0003,  0.0004, -0.0014])
tensor([-0.0002,  0.0006,  0.0005,  ..., -0.0003, -0.0002, -0.0005])
tensor([ 0.0000e+00,  3.0518e-05, -4.8828e-04,  ..., -1.2207e-04,
        -5.4932e-04, -8.5449e-04])
tensor([-0.0004, -0.0005, -0.0005,  ...,  0.0202,  0.0183,  0.0204])
tensor([2.3346e-02, 2.3987e-02, 2.5177e-02,  ..., 3.9673e-04, 0.0000e+00,
        6.1035e-05])
tensor([ 0.0003,  0.0003,  0.0005,  ..., -0.0002, -0.0003,  0.0004])
tensor([ 1.2207e-04, -1.0071e-03,  9.1553e-05,  ...,  3.1616e-02,
         3.3844e-02,  3.3813e-02])
tensor([ 3.2440e-02,  3.1708e-02,  2.9846e-02,  ..., -9.1553e-05,
        -9.1553e-05, -2.7466e-04])
tensor([ 0.0002,  0.0004, -0.0004,  ..., -0.0027, -0.0026, -0.0029])
tensor([-0.0037, -

In [12]:
from sklearn.cluster import AgglomerativeClustering
speaker_count = 2

In [13]:
len(embeddings)
embeddings.shape

(96, 192)

In [24]:
clustering_agg = AgglomerativeClustering(speaker_count).fit(embeddings)

In [25]:
labeled_segments = []
for label, segment in zip(clustering_agg.labels_, result["segments"]):
    labeled_segments.append((label, segment["start"], segment["text"]))

interivew = []
print(INTERVIEW)
for s, (speaker, _, text) in enumerate(labeled_segments):
    stime = INTERVIEW[s]["start"]
    interivew.append(f"話者{speaker + 1} :  {stime} 〜 : 「{text}」")

[{'start': 0.0, 'end': 10.0, 'text': 'どうもこんにちは', 'count': 0}, {'start': 10.0, 'end': 12.0, 'text': 'ありがとうございます', 'count': 0}, {'start': 12.0, 'end': 19.0, 'text': 'ごめん 記者の時間いただきましてどうもありがとうございます', 'count': 0}, {'start': 20.0, 'end': 22.0, 'text': 'おはよう', 'count': 0}, {'start': 24.0, 'end': 29.0, 'text': 'では早速質問をさせていただきたいと思います', 'count': 0}, {'start': 29.0, 'end': 31.0, 'text': 'ちょっとだけお願いします', 'count': 1}, {'start': 31.0, 'end': 49.0, 'text': 'では早速なんですけれども', 'count': 0}, {'start': 49.0, 'end': 55.0, 'text': 'まずフリーをどうでしたでしょうか', 'count': 1}, {'start': 56.0, 'end': 62.0, 'text': '今ちょっと回線がフリーズして聞こえなかったんですけど', 'count': 0}, {'start': 62.0, 'end': 67.0, 'text': 'フリーを導入されたのはいつ頃でしたでしょうか', 'count': 2}, {'start': 67.0, 'end': 79.0, 'text': '今から6年前で2018年の4月ですね', 'count': 0}, {'start': 79.0, 'end': 83.0, 'text': 'すごい そんな前からだったんですね', 'count': 0}, {'start': 83.0, 'end': 85.0, 'text': 'はい', 'count': 0}, {'start': 85.0, 'end': 91.0, 'text': 'その当時他の例えばクラウド会計のものとかって', 'count': 0}, {'start': 91.0, 'end': 93

In [26]:
for speek in interivew:
  print(speek)

話者1 :  0.0 〜 : 「どうもこんにちは」
話者1 :  10.0 〜 : 「ありがとうございます」
話者1 :  12.0 〜 : 「ごめん 記者の時間いただきましてどうもありがとうございます」
話者1 :  20.0 〜 : 「おはよう」
話者1 :  24.0 〜 : 「では早速質問をさせていただきたいと思います」
話者1 :  29.0 〜 : 「ちょっとだけお願いします」
話者1 :  31.0 〜 : 「では早速なんですけれども」
話者1 :  49.0 〜 : 「まずフリーをどうでしたでしょうか」
話者1 :  56.0 〜 : 「今ちょっと回線がフリーズして聞こえなかったんですけど」
話者1 :  62.0 〜 : 「フリーを導入されたのはいつ頃でしたでしょうか」
話者1 :  67.0 〜 : 「今から6年前で2018年の4月ですね」
話者1 :  79.0 〜 : 「すごい そんな前からだったんですね」
話者1 :  83.0 〜 : 「はい」
話者1 :  85.0 〜 : 「その当時他の例えばクラウド会計のものとかって」
話者1 :  91.0 〜 : 「検討されましたでしょうか」
話者2 :  93.0 〜 : 「マネーフォアードとフリーとどっちにしようかなという話はちょっと」
話者2 :  101.0 〜 : 「私の中だけでありましたけども」
話者2 :  105.0 〜 : 「フリーの方がどうも私はその会計に詳しい人間ではないので」
話者1 :  113.0 〜 : 「初心者向きだという話がちょっとしまったのレビューでありましたので」
話者1 :  123.0 〜 : 「じゃあフリーの方いいかなと思ってフリーにしました」
話者1 :  129.0 〜 : 「少し出しました」
話者1 :  131.0 〜 : 「価格とかはその時って検討されましたか企画というか」
話者2 :  138.0 〜 : 「あまり積極的に企画してないんですけど」
話者2 :  145.0 〜 : 「一応フリーの方は」
話者2 :  148.0 〜 : 「いくらだったかな」
話者2 :  150.0 〜 : 「1万2万弱という感じだったので」
話者1 :  156.0 〜 : 「それぐらいならいいかなと思ってそれにしました」
話者1 :  161.0 〜 :

In [21]:
from sklearn import cluster
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
clustering_kmeans = cluster.KMeans(n_clusters=speaker_count)
clustering_kmeans.fit(embeddings)

KMeans(n_clusters=2)

In [28]:
labeled_segments = []
for label, segment in zip(clustering_kmeans.labels_, result["segments"]):
    labeled_segments.append((label, segment["start"], segment["text"]))

interivew = []
print(INTERVIEW)
for s, (speaker, _, text) in enumerate(labeled_segments):
    stime = INTERVIEW[s]["start"]
    interivew.append(f"話者{speaker + 1} :  {stime} 〜 : 「{text}」")

[{'start': 0.0, 'end': 10.0, 'text': 'どうもこんにちは', 'count': 0}, {'start': 10.0, 'end': 12.0, 'text': 'ありがとうございます', 'count': 0}, {'start': 12.0, 'end': 19.0, 'text': 'ごめん 記者の時間いただきましてどうもありがとうございます', 'count': 0}, {'start': 20.0, 'end': 22.0, 'text': 'おはよう', 'count': 0}, {'start': 24.0, 'end': 29.0, 'text': 'では早速質問をさせていただきたいと思います', 'count': 0}, {'start': 29.0, 'end': 31.0, 'text': 'ちょっとだけお願いします', 'count': 1}, {'start': 31.0, 'end': 49.0, 'text': 'では早速なんですけれども', 'count': 0}, {'start': 49.0, 'end': 55.0, 'text': 'まずフリーをどうでしたでしょうか', 'count': 1}, {'start': 56.0, 'end': 62.0, 'text': '今ちょっと回線がフリーズして聞こえなかったんですけど', 'count': 0}, {'start': 62.0, 'end': 67.0, 'text': 'フリーを導入されたのはいつ頃でしたでしょうか', 'count': 2}, {'start': 67.0, 'end': 79.0, 'text': '今から6年前で2018年の4月ですね', 'count': 0}, {'start': 79.0, 'end': 83.0, 'text': 'すごい そんな前からだったんですね', 'count': 0}, {'start': 83.0, 'end': 85.0, 'text': 'はい', 'count': 0}, {'start': 85.0, 'end': 91.0, 'text': 'その当時他の例えばクラウド会計のものとかって', 'count': 0}, {'start': 91.0, 'end': 93

In [32]:
for speek in interivew:
  print(speek)

話者2 :  0.0 〜 : 「どうもこんにちは」
話者2 :  10.0 〜 : 「ありがとうございます」
話者2 :  12.0 〜 : 「ごめん 記者の時間いただきましてどうもありがとうございます」
話者2 :  20.0 〜 : 「おはよう」
話者2 :  24.0 〜 : 「では早速質問をさせていただきたいと思います」
話者2 :  29.0 〜 : 「ちょっとだけお願いします」
話者2 :  31.0 〜 : 「では早速なんですけれども」
話者2 :  49.0 〜 : 「まずフリーをどうでしたでしょうか」
話者2 :  56.0 〜 : 「今ちょっと回線がフリーズして聞こえなかったんですけど」
話者2 :  62.0 〜 : 「フリーを導入されたのはいつ頃でしたでしょうか」
話者1 :  67.0 〜 : 「今から6年前で2018年の4月ですね」
話者2 :  79.0 〜 : 「すごい そんな前からだったんですね」
話者2 :  83.0 〜 : 「はい」
話者2 :  85.0 〜 : 「その当時他の例えばクラウド会計のものとかって」
話者2 :  91.0 〜 : 「検討されましたでしょうか」
話者1 :  93.0 〜 : 「マネーフォアードとフリーとどっちにしようかなという話はちょっと」
話者1 :  101.0 〜 : 「私の中だけでありましたけども」
話者1 :  105.0 〜 : 「フリーの方がどうも私はその会計に詳しい人間ではないので」
話者1 :  113.0 〜 : 「初心者向きだという話がちょっとしまったのレビューでありましたので」
話者2 :  123.0 〜 : 「じゃあフリーの方いいかなと思ってフリーにしました」
話者2 :  129.0 〜 : 「少し出しました」
話者2 :  131.0 〜 : 「価格とかはその時って検討されましたか企画というか」
話者1 :  138.0 〜 : 「あまり積極的に企画してないんですけど」
話者1 :  145.0 〜 : 「一応フリーの方は」
話者1 :  148.0 〜 : 「いくらだったかな」
話者1 :  150.0 〜 : 「1万2万弱という感じだったので」
話者2 :  156.0 〜 : 「それぐらいならいいかなと思ってそれにしました」
話者2 :  161.0 〜 :